# Parser 11.10 - Multiple Argument Functions

This goal of this version of the parser it to teach it to parse expressions containing numeric functions with more than one numeric argument.

In the prevous version of the parser we treated a left parenthesis following a numeric symbol as a binary operator. We could do that because every function we implemented had only one argument. All functions that looked like this:

```Python
fnc(expr)
```

were parsed to Reverse Polish that looked like this:

```Python
fnc expr B(
```

Whenever the evaluator executed the *B(* operator, the top two items on the operand stack were the value of *expr* and the name of *fnc*. It was a simple matter to pop them off and pass control to the appropriate function.

This will no longer do. Consider a function that looks like this:

```Python
fnc(expr1, expr2)
```

We can (and will) treat **','** as an end of expression marker that clears off the operator stack down to (but not including) the left parenthesis that follows *fnc*. But if that is all we do, the final Reverse Polish will look something like:

```Python
fnc expr1 expr2 B(
```

Of course the problem is that neither *expr1* nor *expr2* is a function name. So during evaluation, simply popping the top two items off the operand stack will not get us to the name we need.

One way to solve this is to change the *function call* operator to be unary and arrange for an argument count to appear in the Reverse Polish just before it. The parse of a two argument function might look like:

```Python
fnc expr1 expr2 2 U(
```

When the *function call* operator is executed, the evaluator will pass to it that count as an argument. The operator first pops that number of arguments and collect them in a list. Then one more pop retrieves the function name, which is called indirectly with the list as its argument.

>It is up to the called function to decide what to do with the list.

This mechanism is completely general and will work for any number of arguments, including the one argument functions of the previous version. The simplicity of a single method that works for everything is attractive, but for those existing functions it does mean larger and slower Reverse Polish.

What we will actually do is implement the above scheme only for functions with two or more arguments. We'll create a second *function call* operator to use with these and leave in place everything we already have for single argument functions.

>We are in charge here. We can do anything we want.

## Libraries

In [ ]:
import glob       # for searching directories
import math

import re         # for regular expressions

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, opStk, typStk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', opStk )
        TOstring( 'Type Stack', typStk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Functions

In [ ]:
def fncAbs(val):
    '''absolute value of val'''
    return (True, abs(val))

def fncMax(args):
    '''max of two vals'''
    return (True, max(args))

def fncMin(args):
    '''min of two vals'''
    return (True, min(args))

def fncRnd(val):
    '''rounded val'''
    return (True, round(val))

def fncSgn(val):
    '''sign of val'''
    return (True, 1 if val > 0 else -1 if val < 0 else 0 )

def fncSqt(val):
    '''square root of val'''
    if val >= 0:
        return (True, math.sqrt(val) )
    else:            
        UIerror( 'Negative argument' )
        return ( False, None )
    
# known functions

fncDispatch = {
     'ABS': (fncAbs, 1),
     'MAX': (fncMax, 2),
     'MIN': (fncMin, 2),
   'ROUND': (fncRnd, 1),
    'SIGN': (fncSgn, 1),
    'SQRT': (fncSqt, 1)
}


We add two new functions, *MAX()* and *MIN()*, to demonstrate multiple argument functions. We've also extended the function dictionary to include the number of arguments each function expects.

>*ABS()*, *MAX()* and *MIN()* are merely wrappers for built-in Python functions.

# Parser

In [ ]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar and array numeric variables
# - numeric functions with one or more arguments

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality
# - assignment and shortcut assignment
# - prefix and postfix increment and decrement

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '11.10'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _typeChkLst = {                                         # index is left-to-right, check right-to-left
        'an2_': [  None,   'number', 'argsep' ],
         'n2n': ['number', 'number'],
         'v2n': ['number', 'numsym'],
        'fn2n': ['number', 'fncsym', 'number'],
        'nn2n': ['number', 'number', 'number'],
        'vn2n': ['number', 'numsym', 'number'],
        'vn2v': ['numsym', 'numsym', 'number']
    }
    
    def __init__(self):
        pass
        
    def doparse(self, this):

        def parseErr(mesg, pos):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            if pos > 0:
                UIwriteln(f'{"^^near here".rjust(pos)}')
            return False

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                  # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        opStk = [ ('EOE', 1) ]     # operator stack
        typStk = []                # type stack
        argStk = []                # function argument count stack
        

        def typeCheck(op, chk):
            '''type check operands'''
            nonlocal argStk
            check = list(self._typeChkLst[chk])       # list() to avoid aliasing
            while len(check) > 1:
                want = check.pop()
                
                # are we checking an argument separator ?
                
                if want == 'argsep':
                    if len(opStk) < 2 or opStk[-2][0] != 'B(':     # within a function call ?
                        return parseErr('Unexpected comma', 0)
                    else:
                        argStk[-1] += 1                            # one more argument
                        result.pop()                               # remove 'F,' from RPN
                        continue
                
                have, rpnPos, errPos = typStk.pop()
                
                # no problem ?
                
                if want == have:
                    continue
                
                # did we want a number ?
                
                if want == 'number':
                    
                    # convert symbols to numbers
                    
                    if have == 'numsym':
                        result.insert( rpnPos, 'U*' )
                                           
                # did we want a variable ?
                
                elif want == 'numsym':
                    return parseErr('Numeric variable expected', errPos)
                    
                # did we want a function name ?
                
                elif want == 'fncsym':
                    
                    if have != 'numsym':
                        return parseErr('Function name expected', errPos)
                    fnc = result[rpnPos-1]
                    if not fnc in fncDispatch:
                        return parseErr(f'Unknown function name: {fnc}', errPos)
                    cnt = argStk.pop()
                    if cnt != fncDispatch[fnc][1]:
                        return parseErr(f'Bad argument count: {fnc}', errPos)
                    elif cnt > 1:
                        result.pop()                # remove 'B('
                        result.append(cnt)          # argument count
                        result.append('F()')        # multiple argument function call operator
                    
                # generic everything else
                
                else:
                    return parseErr('Type mismatch', errPos )
                        
            # push result type, RPN operator position, original operator position
                    
            restyp = check.pop()
            if restyp != None:
                typStk.append( (restyp, len(result), errPos) )
            return True
 
        def addOperand(op, typ):
            '''add operand to RPN'''
            result.append( op )
            typStk.append( (typ, len(result), start) )
            
        def addOperator():
            '''add operator to RPN'''            
            op, _, chk = opStk.pop()
            result.append( op )
            return typeCheck( op, chk )
            
        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            ok = True
            while ok and prec <= opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushLeft(op, prec, chk):
            '''push left associative operator on stack'''
            if not popGEop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popGop(prec):
            '''pop operators of greater precedence'''
            ok = True
            while ok and prec < opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushRight(op, prec, chk):
            '''push right associative operator on stack'''
            if not popGop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popUntil(op, prec):
            '''clear and check operator stack'''
            if not popGEop(prec):                       # type check failure ?
                return False

            topop = opStk.pop()[0]                      # found match ?
            if op == topop:
                return True
            
            elif op == '(':
                err = 'right parenthesis'
            elif op == '[':
                err = 'right bracket'
            elif topop == '(':
                err = 'left parenthesis'
            elif topop == '[':
                err = 'left bracket'
            else:
                err = 'EOE'
            
            return parseErr( f'Unmatched {err}', start )
   
        # operator dictionaries initialization
        
        _postOps = '[+]{2}|[-]{2}'
        
        _postOp = {
            '++': (90, pushLeft, 'v2n'),
            '--': (90, pushLeft, 'v2n'),
        }

        _unOps = '!|[+]{1,2}|[-]{1,2}'

        _unOp =  {
            '-':  (80, pushRight, 'n2n'),
            '+':  (80, pushRight, 'n2n'),
            '!':  (80, pushRight, 'n2n'),
            '++': (80, pushRight, 'v2n'),
            '--': (80, pushRight, 'v2n'),
        }

        _binOps = '[-+*/]=?|[=!]?=|\[|\('

        _binOp = {
            '[':  (90, pushLeft,  'vn2v'),
            '(':  (90, pushLeft,  'fn2n'),
            '*':  (70, pushLeft,  'nn2n'),
            '/':  (70, pushLeft,  'nn2n'),
            '+':  (60, pushLeft,  'nn2n'),
            '-':  (60, pushLeft,  'nn2n'),
            '==': (50, pushLeft,  'nn2n'),
            '!=': (50, pushLeft,  'nn2n'),
            '=':  (10, pushRight, 'vn2n'),
            '*=': (10, pushRight, 'vn2n'),
            '/=': (10, pushRight, 'vn2n'),
            '+=': (10, pushRight, 'vn2n'),
            '-=': (10, pushRight, 'vn2n')
        }
      
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range', start)
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            addOperand( uflt, 'numlit' )
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop
              
        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    opStk.append( ('(', 2) )

                elif startsWith(_unOps):
                    '''right unary?'''
                    prec, assoc, check = _unOp[token]
                    assoc( 'U' + token, prec, check )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable or function name?'''
                        addOperand( token.upper(), 'numsym' )
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand', start)

            # look for operator

            else:

                if startsWith('[)\]]'):
                    '''expression terminator?'''
                    ok = popUntil( '(' if token == ')' else '[', 4 )
                        
                    
                elif startsWith(_postOps):
                    '''postfix operator?'''
                    prec, assoc, check = _postOp[token]
                    ok = assoc( 'P' + token, prec, check )
 
                else:
                    
                    wantoperand = True                          # flip

                    if startsWith(_binOps):
                        '''binary operator?'''
                        prec, assoc, check = _binOp[token]
                        ok = assoc( 'B' + token, prec, check )
                        if ok and (token == '[' or token == '('):
                            opStk.append( (token, 2) )
                            if token == '(':
                                argStk.append( 1 )

                    elif startsWith(','):
                        '''function argument separator?'''
                        ok = pushLeft( 'F,', 6, 'an2_' )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator', start)

            PTshowparse(ok, result, opStk, typStk )               # trace
 
        if ok and wantoperand:
            ok = parseErr('Unexpected end of expression', start ) # must be in 'wantoperator' state   

        if ok:
            ok = popUntil( 'EOE', 3 )                             # clear operator stack
            
        if ok:
            ok = typeCheck('LoneSymbol', 'n2n')                   # make sure final result is a number

        return (ok, result if ok else None)                       # done
       

### How it works

The first thing we need is a way to count the actual number of arguments any function call has. This is the purpose of *argStk[]*. We make it a stack so we can properly handle nested function calls.

If we find a function left parenthesis we do as before, except we also initialize the current nesting level of *argStk[]* to one.

The argument separator **','** (a comma) is in fact an expression terminator. It would be nice to handle this along with the **')'** and **']'** expression terminators, but unlike them it is more like an operator. We need to switch state after finding one. It seems cleaner just to provide a separate check.

When we do find an argument separator, we treat it as a low precedence, left associative operator. When it is pushed onto the operator stack, it will clear the stack down to the left parenthesis protecting the *B(* operator. It will stay there until popped off by another argument separator or the matching right parenthesis.

Since a comma is legal only in the context of a function call, we must check to make sure. We can do this either as soon as we find it or wait until it is type checked. If we check right away it is easier to indicate just where the error happened. However even if we do that, we still have to perform a type check later on the argument expression. It's something of a matter of taste, but here we'll try to do all the checking we need to within the type check.

When the comma is popped off, it will have a new kind of type check, *'an2_'*, "argument separator and number to nothing". This is essentially a special case. At the point it happens, a function with two arguments looks in the Reverse Polish like:

```Python
fncname expr1 expr2 F,
```

the operator stack looks like:

```Python
B( (
```

the type stack looks like:

```Python
number number
```

and the top element in the argument count stack looks like:

```Python
1
```

...at least for a syntactically correct expression.

Note that since only operands and the results of operators can put anything on the type stack, and *F,* doesn't actually do anything to any operand, there is nothing on the type stack it can check. We can't pull anything off the type stack to test (or if we did, we'd have to push everything right back on). But because the precedence of *F,* is just higher than a left parenthesis, after it is popped off the operator stack the top two operators left on it should be a left parenthesis protecting a *B(* operator. If we don't find that, the comma did not appear within a function call.

If we are within a function call, we increment the top item on the argument count stack. It is also convenient to remove *F,* from the Reverse Polish, since it serves a purpose only during parsing. Then we re-start the type check loop.

If the loop is exited normally, no result type is pushed on the type stack. There isn't one. An argument separator doesn't actually do anything to any operand.

We're not quite done with modifying *typeCheck()*. Checking the *B(* operator requires some additional work. After verifying that the function name exists, the number of arguments actually provided is compared to the number specified in the function dictionary. If the count is more than one, the *B(* operator is replaced by the count and a new operator we will use to call any function with multiple arguments, *F()*, is added to the Reverse Polish:

```Python
fnc expr1 expr2 ... exprN B(  =>  fnc expr1 expr2 ... exprN N F()
```

This mechanism is completely general and works for any number of arguments, including the single argument functions of the previous version. We *could* use this for all function calls.

But we can also save a little space in the Reverse Polish by allowing ourselves *two* function call operators. One for single argument functions, *B(*, and one for multiple argument functions, *F()*. Which is what we'll do. If there is only one argument, we won't do anything to the existing Reverse Polish at all.

# Evaluator

In [ ]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference
# - variable assignment and shortcut assignment
# - prefix and postfix increment and decrement
# - numeric array assignment and de-reference
# - function calls with one or more arguments

# errors detected:
# - out of range
# - division by zero
# - invalid function arguments

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        self._symTable = dict()
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
        
        def binAryNdx(ndx, nam):
            '''create array index'''
            return inRange( 0 <= ndx <= self._FLTMAX, f'{nam}_{ndx}' )
                     
        def binVal(val, var):
            '''assign value to variable'''
            self._symTable[var] = val
            return pushOperand(val)
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(self._symTable[var] if var in self._symTable else 0)

#        def unPlu(arg):
#           '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )
 
        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )
 
        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            elif lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft * rgt )

            elif rgt > 0:
                return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
  
            else:
                return inRange( rgt >= self._FLTMAX / lft, lft * rgt )
 
        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            elif rgt > 0:
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
            
            elif rgt < 0:
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        def binShortVal(val, var, op):
            '''shortcut assignment'''
            _ = unVal(var)                           # put the value of the variable on the stack
            if op(val, stk.pop()):                   # perform the arithmetic
                return binVal(stk.pop(), var)        # if successful,  also assign result to variable
            return False
        
        def binAddVal(val, var):
            return binShortVal(val, var, binAdd)
        
        def binSubVal(val, var):
            return binShortVal(val, var, binSub)
        
        def binMulVal(val, var):
            return binShortVal(val, var, binMul)
        
        def binDivVal(val, var):
            return binShortVal(val, var, binDiv)
        
        def unPfxInc(var):
            return binShortVal(1, var, binAdd)
        
        def unPfxDec(var):
            return binShortVal(1, var, binSub)
        
        def unPostFix(val, var):
            '''postfix inc/dec'''
            _ = unVal(var)                             # push current value of variable on stack
            ok = binShortVal(val, var, binAdd)         # update value of variable
            stk.pop()                                  # remove updated value from stack (if error, removes first push)
            return ok
                
        def unPstInc(var):
            return unPostFix(1, var)
            
        def unPstDec(var):
            return unPostFix(-1, var)
        
        def binFnc(val, fnc):
            '''single argument function call'''
            ok, val = fncDispatch[fnc][0](val)
            return pushOperand(val) if ok else False
                
        def multiFnc(cnt):
            '''multiple argument function call'''
            args = []
            while cnt > 0:
                args.append( stk.pop() )
                cnt -= 1
            ok, val = fncDispatch[stk.pop()][0](args)               
            return pushOperand(val) if ok else False
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot,
            'U++': unPfxInc,
            'U--': unPfxDec,
            'P++': unPstInc,
            'P--': unPstDec,
            'F()': multiFnc
        }
        
        binDispatch = {
            'B+': binAdd, 'B-': binSub,
            'B+=': binAddVal, 'B-=': binSubVal,
            'B*': binMul, 'B/': binDiv,
            'B*=': binMulVal, 'B/=': binDivVal,
            'B==': binEqu, 'B!=': binNeq,
            'B=': binVal,
            'B[': binAryNdx, 'B(': binFnc
        }
  
        stk = []
        ok = True

        # main loop
        
        TOstring('Input', rpn)
        
        for v in rpn:

            ETshowtoken(v)
            
            if v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

We add *F()* to *unDispatch{}*. *multiFnc()* receives the number of arguments to pass to the named function. Each is popped (in right-to-left order) and appended to a list which is passed as a whole. The named function can handle this list however it likes. If the returned value is valid, it is pushed on the operand stack, otherwise evaluatioh is halted.

## Running the parser

In [ ]:
passCnt = failCnt = 0                          # most useful for test input files, but never any harm

myParser = myEvaluator = None                  # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser
        
def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    neg = this[0] == '@'
    if neg:
        this = this[1:]
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
        if neg:
            ok = not ok
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [ ]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    myEvaluator._symTable.clear()
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests